In [1]:
import glob
import os
import pandas as pd
import shutil

In [2]:
downloadedDir='/Users/irffanalahi/Desktop/noneed/TCGA'
outdir=downloadedDir+"_all_info"
os.makedirs(outdir)

In [3]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [4]:
downloadedDirfiles=listdir_nohidden(downloadedDir)
downloadedDirfiles

['/Users/irffanalahi/Desktop/noneed/TCGA/Skin',
 '/Users/irffanalahi/Desktop/noneed/TCGA/illumina_retrieve_pairwise.py',
 '/Users/irffanalahi/Desktop/noneed/TCGA/Larynx',
 '/Users/irffanalahi/Desktop/noneed/TCGA/Brain']

In [5]:
alldirs=[]
for adriname in downloadedDirfiles:
    if os.path.isdir(adriname):
        alldirs.append(adriname)
alldirs

['/Users/irffanalahi/Desktop/noneed/TCGA/Skin',
 '/Users/irffanalahi/Desktop/noneed/TCGA/Larynx',
 '/Users/irffanalahi/Desktop/noneed/TCGA/Brain']

In [6]:
for adir in alldirs:
    infoname=adir+"/"+os.path.basename(adir)+".txt"
    if os.path.exists(infoname)==True:
        shutil.copy(infoname, outdir+"/"+os.path.basename(adir)+".txt")

In [7]:
files = glob.glob(outdir+'/*.txt')
filesdf=[pd.read_csv(fp,sep="\t") for fp in files]

In [8]:
df_combined = pd.concat(filesdf, axis=0)
df_combined.head()

,cases.0.disease_type,cases.0.primary_site,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.submitter_id,file_name,id
0,Gliomas,Brain,TCGA-GBM,Primary Tumor,TCGA-74-6573,aa287e61-7e64-4f01-9e44-606abd9ac93a.methylati...,dbc50065-31b1-42ce-ab81-7e8388befdfa
1,Gliomas,Brain,TCGA-GBM,Solid Tissue Normal,TCGA-74-6573,07d27a99-baf0-4781-bd44-641303f93ead.methylati...,54f1b1e0-bba7-417d-9b57-8c6ffc332845
0,Nevi and Melanomas,Skin,TCGA-SKCM,Solid Tissue Normal,TCGA-FW-A3R5,875a9646-fef4-45ed-836b-123c2a9b0037.methylati...,3a73da68-4999-4971-b0c5-926a032f50e2
1,Nevi and Melanomas,Skin,TCGA-SKCM,Metastatic,TCGA-FW-A3R5,2222a810-9fd4-4dcf-8fa9-d5a6e5d9afaf.methylati...,4bd4daef-4ce5-448c-aa4e-e5e20b38a6c2
2,Nevi and Melanomas,Skin,TCGA-SKCM,Metastatic,TCGA-GN-A4U8,8ca8eb5c-c4da-4475-a2ef-9e8a61e2bbdd.methylati...,1beda168-eaed-4914-80a8-5e8cf588f166


In [9]:
df_combined_rename=df_combined.copy()

In [10]:
df_combined_rename.rename(columns={"cases.0.disease_type": "Disease type", "cases.0.primary_site": "Primary site",'cases.0.project.project_id':'Project','cases.0.samples.0.sample_type':'Sample type','cases.0.submitter_id':'Submitter id','file_name':'File name','id':'File UUID'},inplace=True)
df_combined_rename.head()

,Disease type,Primary site,Project,Sample type,Submitter id,File name,File UUID
0,Gliomas,Brain,TCGA-GBM,Primary Tumor,TCGA-74-6573,aa287e61-7e64-4f01-9e44-606abd9ac93a.methylati...,dbc50065-31b1-42ce-ab81-7e8388befdfa
1,Gliomas,Brain,TCGA-GBM,Solid Tissue Normal,TCGA-74-6573,07d27a99-baf0-4781-bd44-641303f93ead.methylati...,54f1b1e0-bba7-417d-9b57-8c6ffc332845
0,Nevi and Melanomas,Skin,TCGA-SKCM,Solid Tissue Normal,TCGA-FW-A3R5,875a9646-fef4-45ed-836b-123c2a9b0037.methylati...,3a73da68-4999-4971-b0c5-926a032f50e2
1,Nevi and Melanomas,Skin,TCGA-SKCM,Metastatic,TCGA-FW-A3R5,2222a810-9fd4-4dcf-8fa9-d5a6e5d9afaf.methylati...,4bd4daef-4ce5-448c-aa4e-e5e20b38a6c2
2,Nevi and Melanomas,Skin,TCGA-SKCM,Metastatic,TCGA-GN-A4U8,8ca8eb5c-c4da-4475-a2ef-9e8a61e2bbdd.methylati...,1beda168-eaed-4914-80a8-5e8cf588f166


In [11]:
grouped=df_combined_rename.groupby('Project')

In [12]:
from collections import defaultdict

summarydict=defaultdict(list)

for name, group in grouped:
    normal=group[group['Sample type']=='Solid Tissue Normal'].shape[0]
    tumor=group.shape[0]-normal
    summarydict['Project'].append(name)
    summarydict['Normal'].append(normal)
    summarydict['Tumor'].append(tumor)
    
    

In [13]:
summarydf=pd.DataFrame.from_dict(summarydict)
summarydf

,Project,Normal,Tumor
0,TCGA-GBM,1,1
1,TCGA-SKCM,2,2


In [14]:
outexcel=df_combined_rename.drop(['Disease type','Submitter id'],axis=1)
outexcel=outexcel[['Project','Primary site','Sample type','File name','File UUID']]
outexcel.head()

,Project,Primary site,Sample type,File name,File UUID
0,TCGA-GBM,Brain,Primary Tumor,aa287e61-7e64-4f01-9e44-606abd9ac93a.methylati...,dbc50065-31b1-42ce-ab81-7e8388befdfa
1,TCGA-GBM,Brain,Solid Tissue Normal,07d27a99-baf0-4781-bd44-641303f93ead.methylati...,54f1b1e0-bba7-417d-9b57-8c6ffc332845
0,TCGA-SKCM,Skin,Solid Tissue Normal,875a9646-fef4-45ed-836b-123c2a9b0037.methylati...,3a73da68-4999-4971-b0c5-926a032f50e2
1,TCGA-SKCM,Skin,Metastatic,2222a810-9fd4-4dcf-8fa9-d5a6e5d9afaf.methylati...,4bd4daef-4ce5-448c-aa4e-e5e20b38a6c2
2,TCGA-SKCM,Skin,Metastatic,8ca8eb5c-c4da-4475-a2ef-9e8a61e2bbdd.methylati...,1beda168-eaed-4914-80a8-5e8cf588f166


In [15]:

with pd.ExcelWriter(outdir+'.xlsx') as writer:  
    outexcel.to_excel(writer, sheet_name='Sheet_1',index=False)
    summarydf.to_excel(writer, sheet_name='summary',index=False)

In [16]:
df_combined.to_excel(outdir+"_rawinfo.xlsx",index=False)